In [1]:
import pandas as pd
from collections import Counter

# Reading and cleaning dataset

In [2]:
df = pd.read_csv("/content/grocery_data_mar_2025.csv")

In [3]:
df.columns

Index(['productId', 'productImage', 'brand', 'title', 'description',
       'packageSizing', 'deal', 'mopDeal', 'pcoDeal', 'inventoryIndicator',
       'offerType', 'offerId', 'action', 'isVariant', 'link', 'productVendor',
       'productVariant', 'textBadge', 'variantTheme', 'articleNumber', 'uom',
       'sponsoredCreative', 'isSponsored', 'isComplementarySponsored',
       'complementarySponsoredBadgeText', 'productBadge', 'pricing.price',
       'pricing.wasPrice', 'pricing.displayPrice', 'pricing.memberOnlyPrice',
       'pricing.mopDisplayPrice', 'pricing.ehfTotal',
       'pricing.environmentalHandlingFee', 'pricingUnits.type',
       'pricingUnits.unit', 'pricingUnits.interval',
       'pricingUnits.minOrderQuantity', 'pricingUnits.maxOrderQuantity',
       'pricingUnits.weighted', 'deal.type', 'deal.text', 'deal.points',
       'deal.name', 'deal.expiryDate', 'deal.dealPrice',
       'inventoryIndicator.indicatorId', 'inventoryIndicator.text',
       'productBadge.badgeId', '

In [4]:
df = df[['brand','title','packageSizing','pricing.price']]

### Converting column values

In [5]:
df = df.copy()
df["title"] = df["title"].str.lower()
df['brand'] = df['brand'].str.lower()
df['pricing.price'] = df['pricing.price'].astype(float)
df = df.drop_duplicates()
df

,brand,title,packageSizing,pricing.price
0,NaN,roma tomatoes,$7.69/1kg $3.49/1lb,0.92
1,NaN,cilantro,"1 ea, $0.99/1ea",0.50
2,NaN,mango,"1 ea, $1.49/1ea",1.49
3,NaN,ginger,$3.26/1kg $1.48/1lb,0.68
4,rooster,"garlic bulbs, 3-count","90 g, $0.33/1ea",1.00
...,...,...,...,...
10021,wholesome sweeteners,whole organic raw blue agave syrup,"480 ml, $2.08/100ml",9.99
10022,a&w,diet root beer,"12x355.0 ml, $0.19/100ml",6.99
10024,castello,"cream cheese spread, sweet & spicy peppers","125 g, $4.00/100g",5.00
10025,NaN,"jelly crystals, mixed fruit","90 g, $1.43/100g",1.29


### Splitting packageSizing into Item size and Cost per unit

- Further split cost per unit into the price amount and the unit metric

In [6]:
# Function to split based on ',' if present, otherwise split on whitespace
def custom_split(value):
    if ',' in value:
        return value.split(',', 1)  # Split on first comma
    else:
        return value.split(maxsplit=1)  # Split on first whitespace

# Apply the function and expand into two new columns
df[['item_total_sizing', 'cost_per_unit']] = df['packageSizing'].apply(lambda x: pd.Series(custom_split(x)))
df[['price_amount', 'price_unit']] = df['cost_per_unit'].str.extract(r'\$([\d.]+)\/(\S+)')
df

,brand,title,packageSizing,pricing.price,item_total_sizing,cost_per_unit,price_amount,price_unit
0,NaN,roma tomatoes,$7.69/1kg $3.49/1lb,0.92,$7.69/1kg,$3.49/1lb,3.49,1lb
1,NaN,cilantro,"1 ea, $0.99/1ea",0.50,1 ea,$0.99/1ea,0.99,1ea
2,NaN,mango,"1 ea, $1.49/1ea",1.49,1 ea,$1.49/1ea,1.49,1ea
3,NaN,ginger,$3.26/1kg $1.48/1lb,0.68,$3.26/1kg,$1.48/1lb,1.48,1lb
4,rooster,"garlic bulbs, 3-count","90 g, $0.33/1ea",1.00,90 g,$0.33/1ea,0.33,1ea
...,...,...,...,...,...,...,...,...
10021,wholesome sweeteners,whole organic raw blue agave syrup,"480 ml, $2.08/100ml",9.99,480 ml,$2.08/100ml,2.08,100ml
10022,a&w,diet root beer,"12x355.0 ml, $0.19/100ml",6.99,12x355.0 ml,$0.19/100ml,0.19,100ml
10024,castello,"cream cheese spread, sweet & spicy peppers","125 g, $4.00/100g",5.00,125 g,$4.00/100g,4.00,100g
10025,NaN,"jelly crystals, mixed fruit","90 g, $1.43/100g",1.29,90 g,$1.43/100g,1.43,100g


### Renaming the column for better understanding by the Gemini Model

In [7]:
df = df.rename(columns={
    'title': 'product_name',
    'packageSizing': 'size_options',
    'pricing.price': 'base_price',
    'item_total_sizing': 'product_size',
    'cost_per_unit': 'price_per_unit',
    'price_amount': 'unit_price_value',
    'price_unit': 'unit_measure'
})
df = df.drop(columns=['size_options'])

#### Converting columns to numeric column type and check the `dtypes` of columns

In [8]:
df['base_price'] = pd.to_numeric(df['base_price'])
df['unit_price_value'] = pd.to_numeric(df['unit_price_value'])
df.dtypes

,0
brand,object
product_name,object
base_price,float64
product_size,object
price_per_unit,object
unit_price_value,float64
unit_measure,object


In [9]:
df.head(5)

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
0,NaN,roma tomatoes,0.92,$7.69/1kg,$3.49/1lb,3.49,1lb
1,NaN,cilantro,0.50,1 ea,$0.99/1ea,0.99,1ea
2,NaN,mango,1.49,1 ea,$1.49/1ea,1.49,1ea
3,NaN,ginger,0.68,$3.26/1kg,$1.48/1lb,1.48,1lb
4,rooster,"garlic bulbs, 3-count",1.00,90 g,$0.33/1ea,0.33,1ea


#### Top 20 brand names present in the dataset

In [10]:
df['brand'].value_counts().head(20)

,count
brand,
president's choice,781
no name,353
pc organics,136
pc blue menu,83
club house,69
farmer's market,62
suraj,59
christie,59
t&t,51


#### Functions used in data cleaning

In [11]:
# Returns a DataFrame containing the searched word, sorted by the `base_price`
def food_item(w):
  curated_item_df = df[df['product_name'].str.contains(w, case=False, na=False)]
  curated_item_df = curated_item_df.sort_values(by='base_price')

  return curated_item_df

# Returns the word count for the words present in the DataFrame under the `product_name` column
def word_counts(tf):
  words = tf['product_name'].str.split()
  all_words = [word.lower() for sublist in words for word in sublist]

  word_counts = Counter(all_words)
  word_counts_df = pd.DataFrame(word_counts.items(), columns=['Word', 'Count']).sort_values(by='Count', ascending=False)

  return word_counts_df

# Returns rows from the DataFrame where the 'brand' column contains any of the words in the word list
def exclude_brand_rows(df, word_list):
  pattern = '|'.join(word_list)
  filtered_df = df[~df['brand'].str.contains(pattern, case=False, na=False)]

  return filtered_df

# Returns rows from the DataFrame where the 'product_name' column contains any of the words in the word list
def exclude_product_rows(df, word_list):
  pattern = '|'.join(word_list)
  filtered_df = df[~df['product_name'].str.contains(pattern, case=False, na=False)]

  return filtered_df

# Returns rows from the DataFrame where the 'product_name' column contains any of the words in the word list
def include_product_rows(df, word_list, item_name):
  pattern = '|'.join(word_list)
  filtered_df = df[df['product_name'].str.contains(pattern, case=False, na=False)]
  filtered_df = filtered_df.copy()
  filtered_df['item'] = item_name

  return filtered_df

# Returns two DataFrames vertically (row-wise) and resets the index.
def combine_dataframes(df1, df2):
  df1 = df1.reset_index(drop=True)
  df2 = df2.reset_index(drop=True)

  # Combine vertically and ignore original indices
  combined_df = pd.concat([df1, df2], axis=0, ignore_index=True)

  return combined_df

#### Brand Removal which are not useful for this use case

In [12]:
word_list = ['lindt','betty crocker', 'chapmans', 'nestl', 'campbell', 'haldiram', 'knorr', 'starbucks', 'cadbury', 'hershey', 'kinder', 'kraft', 'johnvince foods',
             'general mills', 'burt', 'tim hortons', 'club house', 'kraft']

df = exclude_brand_rows(df, word_list)
df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
0,NaN,roma tomatoes,0.92,$7.69/1kg,$3.49/1lb,3.49,1lb
1,NaN,cilantro,0.50,1 ea,$0.99/1ea,0.99,1ea
2,NaN,mango,1.49,1 ea,$1.49/1ea,1.49,1ea
3,NaN,ginger,0.68,$3.26/1kg,$1.48/1lb,1.48,1lb
4,rooster,"garlic bulbs, 3-count",1.00,90 g,$0.33/1ea,0.33,1ea
...,...,...,...,...,...,...,...
10021,wholesome sweeteners,whole organic raw blue agave syrup,9.99,480 ml,$2.08/100ml,2.08,100ml
10022,a&w,diet root beer,6.99,12x355.0 ml,$0.19/100ml,0.19,100ml
10024,castello,"cream cheese spread, sweet & spicy peppers",5.00,125 g,$4.00/100g,4.00,100g
10025,NaN,"jelly crystals, mixed fruit",1.29,90 g,$1.43/100g,1.43,100g


# Food Items

#### 1. Eggs

In [13]:
word = 'egg'
eggs_df = food_item(word)
eggs_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
2857,t&t,chinese pastry with egg,0.99,228 g,$0.43/100g,0.43,100g
2942,sunrype,fruitsource apple wildberry carrot 100% fruit ...,1.69,37 g,$4.57/100g,4.57,100g
1082,monde,eggnog cookies,1.79,130 g,$1.38/100g,1.38,100g
8539,rooster,thick shrimp egg noodles,2.29,454 g,$0.50/100g,0.50,100g
20,NaN,purple eggplants,2.47,$4.34/1kg,$1.97/1lb,1.97,1lb
...,...,...,...,...,...,...,...
2736,tuscanini,"kosher premium pizza, grilled veggie",11.99,425 g,$2.82/100g,2.82,100g
9738,NaN,"veggie tray, tri-pack",12.00,1.01 kg,$1.19/100g,1.19,100g
9769,NaN,"veggie platter, small (broccoli, carrots, mini...",14.00,962 g,$1.46/100g,1.46,100g
3644,jean talon,lol super egg,14.99,150 g,$9.99/100g,9.99,100g


In [14]:
eggs_df['brand'].value_counts().head(10)

,count
brand,
kelloggs,9
sensible portions,6
no name,6
president's choice,6
yves,6
burnbrae farms,5
pc blue menu,4
pc organics,4
sau tao,3


#### Brand Removal for eggs which are not useful

In [15]:
brand_list = ['cadbury', 'kinder', 'lindt', 'hershey', 'rooster', 't&t', 'hellman','kellogg','reese','veeba']
eggs_df = exclude_brand_rows(eggs_df, brand_list)

In [16]:
eggs_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
2942,sunrype,fruitsource apple wildberry carrot 100% fruit ...,1.69,37 g,$4.57/100g,4.57,100g
1082,monde,eggnog cookies,1.79,130 g,$1.38/100g,1.38,100g
20,NaN,purple eggplants,2.47,$4.34/1kg,$1.97/1lb,1.97,1lb
3489,dare,"sour jelly eggs, spring mix",2.49,225 g,$1.11/100g,1.11,100g
8466,dare,breton veggie bites,2.49,200 g,$1.75/100g,1.75,100g
...,...,...,...,...,...,...,...
2736,tuscanini,"kosher premium pizza, grilled veggie",11.99,425 g,$2.82/100g,2.82,100g
9738,NaN,"veggie tray, tri-pack",12.00,1.01 kg,$1.19/100g,1.19,100g
9769,NaN,"veggie platter, small (broccoli, carrots, mini...",14.00,962 g,$1.46/100g,1.46,100g
3644,jean talon,lol super egg,14.99,150 g,$9.99/100g,9.99,100g


#### Checking the count of words present in `product_name`

In [17]:
eggs_word_df = word_counts(eggs_df)
eggs_word_df.head(10)

,Word,Count
7,veggie,32
26,egg,24
15,"eggs,",13
64,&,13
45,eggs,12
54,large,10
79,white,8
85,with,7
29,garden,6
57,veggies,6


### Word Removal

In [18]:
word_list = ['veggie','chocolate','cocoa','easter','filling','waffle','noodles','eggnog','wrap','breakfast','salad','mayo','eggplant','jelly','candy','pickle']
eggs_df = exclude_product_rows(eggs_df, word_list)
eggs_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
2104,pc blue menu,blue menu peeled hard boiled omega-3 free run ...,3.39,3x88.0 g,$3.85/100g,3.85,100g
9003,kosa,quail eggs in water,3.99,425 g,$0.94/100g,0.94,100g
889,no name,"eggs, medium",4.06,12 ea,$0.34/1ea,0.34,1ea
2915,no name,"eggs, large",4.16,12 ea,$0.35/1ea,0.35,1ea
5859,pc blue menu,free run egg whites,5.29,500 g,$1.06/100g,1.06,100g
3362,no name,grade a large brown eggs,5.37,12 ea,$0.45/1ea,0.45,1ea
1606,burnbrae farms,naturegg simply egg whites,5.39,1 ea,$1.08/100ml,1.08,100ml
2506,burnbrae farms,"egg creations, original",5.49,500 g,$1.10/100g,1.10,100g
4784,burnbrae farms,"naturegg nest laid white eggs, large",5.69,12 ea,$0.47/1ea,0.47,1ea
7356,golden valley,"born 3 white eggs, large",5.99,12 ea,$0.50/1ea,0.50,1ea


In [19]:
eggs_df = eggs_df[eggs_df['product_size'].str.contains('ea')]

Final list for eggs

In [20]:
word_list = ['eggs, large', 'grade a large brown eggs', 'naturegg simply egg whites', 'white eggs, extra large', 'large brown eggs']
eggs_df = include_product_rows(eggs_df, word_list, 'eggs')
eggs_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure,item
2915,no name,"eggs, large",4.16,12 ea,$0.35/1ea,0.35,1ea,eggs
3362,no name,grade a large brown eggs,5.37,12 ea,$0.45/1ea,0.45,1ea,eggs
1606,burnbrae farms,naturegg simply egg whites,5.39,1 ea,$1.08/100ml,1.08,100ml,eggs
4784,burnbrae farms,"naturegg nest laid white eggs, large",5.69,12 ea,$0.47/1ea,0.47,1ea,eggs
7356,golden valley,"born 3 white eggs, large",5.99,12 ea,$0.50/1ea,0.50,1ea,eggs
5870,burnbrae farms,"naturegg omega 3 white eggs, large",6.49,12 ea,$0.54/1ea,0.54,1ea,eggs
3688,golden valley,"white eggs, extra large",7.07,18 ea,$0.39/1ea,0.39,1ea,eggs
41,pc organics,free range large brown eggs,7.99,12 ea,$0.67/1ea,0.67,1ea,eggs
598,pc organics,"free-range large brown eggs, club pack (30 count)",16.99,30 ea,$0.57/1ea,0.57,1ea,eggs


#### 2. Chicken

In [21]:
word = 'chicken'
chicken_df  = food_item(word)
chicken_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
5694,no name,chicken flavour instant noodle soup,0.39,85 g,$0.46/100g,0.46,100g
3642,lucky me!,instant noodle soup artificial chicken flavour,0.79,55 g,$1.44/100g,1.44,100g
9397,bens,fast & fancy homestyle chicken flavour & wild ...,0.97,132 g,$1.43/100g,1.43,100g
1574,no name,condensed chicken noodle soup,1.00,284 ml,$0.35/100ml,0.35,100ml
9702,aylmer,chicken noodle soup,1.00,284 ml,$0.42/100ml,0.42,100ml
...,...,...,...,...,...,...,...
465,president's choice,club pack fully cooked buttermilk chicken brea...,35.99,1.81 kg,$1.99/100g,1.99,100g
4494,no name,seasoned chicken breasts club size,38.99,2 kg,$1.95/100g,1.95,100g
3386,NaN,split chicken wings,39.00,4 kg,$0.98/100g,0.98,100g
5891,no name,"club pack chicken breasts, boneless skinless",39.99,2 kg,$2.00/100g,2.00,100g


In [22]:
top_brands = chicken_df['brand'].value_counts().head(7).index.tolist()
word_list = ['knorr', 'ziggy', 'janes', 'swanson']
chicken_df = exclude_brand_rows(chicken_df, word_list)

In [23]:
chicken_df['brand'].value_counts()

,count
brand,
president's choice,43
no name,21
zabiha halal,10
pc blue menu,10
swiss chalet,5
...,...
freybe,1
kebab factory,1
el monterey,1


In [24]:
chicken_word_df = word_counts(chicken_df)
chicken_word_df.head(20)

,Word,Count
0,chicken,222
7,&,22
106,breast,18
4,soup,17
3,noodle,15
152,breaded,14
193,club,14
186,pack,14
43,with,13
35,butter,12


In [25]:
word_list = ['soup','noodle','flavour','with','breaded','butter','cooked','fully','rice','broth','mix','style','frozen']
chicken_df = exclude_product_rows(chicken_df, word_list)

In [26]:
word_list = ['chicken sausages', 'extra lean ground chicken', 'halal whole chicken', 'club pack chicken thighs, bone-in', 'chicken wings, club pack', 'club pack chicken wings, split', 'club pack chicken breasts, boneless skinless']
chicken_df = include_product_rows(chicken_df, word_list, 'chicken')
chicken_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure,item
7755,zabiha halal,chicken sausages,6.99,375 g,$1.86/100g,1.86,100g,chicken
3052,sufra,extra lean ground chicken,8.00,454 g,$1.76/100g,1.76,100g,chicken
6461,sufra,halal whole chicken,14.00,1.7 kg,$0.82/100g,0.82,100g,chicken
7802,no name,"club pack chicken thighs, bone-in",17.99,2 kg,$0.90/100g,0.90,100g,chicken
7180,no name,"honey garlic chicken wings, club pack",26.50,1.81 kg,$1.46/100g,1.46,100g,chicken
7304,no name,"chicken wings, club pack",26.50,1.81 kg,$1.46/100g,1.46,100g,chicken
4782,no name,"club pack chicken wings, split",26.99,2 kg,$1.35/100g,1.35,100g,chicken
5891,no name,"club pack chicken breasts, boneless skinless",39.99,2 kg,$2.00/100g,2.00,100g,chicken


Merging Eggs and Chicken dataframe

In [27]:
final_df = combine_dataframes(eggs_df, chicken_df)
final_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure,item
0,no name,"eggs, large",4.16,12 ea,$0.35/1ea,0.35,1ea,eggs
1,no name,grade a large brown eggs,5.37,12 ea,$0.45/1ea,0.45,1ea,eggs
2,burnbrae farms,naturegg simply egg whites,5.39,1 ea,$1.08/100ml,1.08,100ml,eggs
3,burnbrae farms,"naturegg nest laid white eggs, large",5.69,12 ea,$0.47/1ea,0.47,1ea,eggs
4,golden valley,"born 3 white eggs, large",5.99,12 ea,$0.50/1ea,0.50,1ea,eggs
5,burnbrae farms,"naturegg omega 3 white eggs, large",6.49,12 ea,$0.54/1ea,0.54,1ea,eggs
6,golden valley,"white eggs, extra large",7.07,18 ea,$0.39/1ea,0.39,1ea,eggs
7,pc organics,free range large brown eggs,7.99,12 ea,$0.67/1ea,0.67,1ea,eggs
8,pc organics,"free-range large brown eggs, club pack (30 count)",16.99,30 ea,$0.57/1ea,0.57,1ea,eggs
9,zabiha halal,chicken sausages,6.99,375 g,$1.86/100g,1.86,100g,chicken


#### 3. Pork

Taking food items having pork in `product_name` with `base_price` more than 5 as below it would consist of unuseful items

In [28]:
word = 'pork'
pork_df = df[df['product_name'].str.contains(word, case=False, na=False)]
pork_df = pork_df[pork_df['base_price'] >= 5]
pork_df = pork_df.sort_values(by='base_price')
pork_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
5649,t&t,pork & leek potstickers,5.00,567 g,$0.88/100g,0.88,100g
4163,NaN,cured salt pork,5.23,$19.82/1kg,$8.99/1lb,8.99,1lb
6284,siwin,pork tocino,5.49,375 g,$1.46/100g,1.46,100g
3265,president's choice,pork & chicken liver pâté with black pepper,5.49,150 g,$3.66/100g,3.66,100g
7774,president's choice,free from lean ground pork,5.49,454 g,$1.21/100g,1.21,100g
...,...,...,...,...,...,...,...
6237,NaN,"lethbridge pork butt roast, boneless",17.96,$17.61/1kg,$7.99/1lb,7.99,1lb
4509,mitchell's,pure pork sausages,22.99,2 kg,$1.15/100g,1.15,100g
40,NaN,"pork back ribs, 2-pack",33.22,$15.41/1kg,$6.99/1lb,6.99,1lb
4651,NaN,frozen pork bone-in half loin bagged,43.16,$8.80/1kg,$3.99/1lb,3.99,1lb


In [29]:
pork_word_df = word_counts(pork_df)
pork_word_df.head(10)

,Word,Count
0,pork,61
19,sausages,14
1,&,9
49,lethbridge,6
32,italian,6
50,loin,6
13,free,5
14,from,5
51,centre,4
34,buns,4


In [30]:
word_list = ['italian','garlic','hot','bbq','flavour']
pork_df = exclude_product_rows(pork_df, word_list)

In [31]:
word_list = ['free from lean ground pork', 'pork loin half boneless frozen', 'medium ground pork']
pork_df = include_product_rows(pork_df, word_list, 'pork')

In [32]:
final_df = combine_dataframes(final_df, pork_df)
final_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure,item
0,no name,"eggs, large",4.16,12 ea,$0.35/1ea,0.35,1ea,eggs
1,no name,grade a large brown eggs,5.37,12 ea,$0.45/1ea,0.45,1ea,eggs
2,burnbrae farms,naturegg simply egg whites,5.39,1 ea,$1.08/100ml,1.08,100ml,eggs
3,burnbrae farms,"naturegg nest laid white eggs, large",5.69,12 ea,$0.47/1ea,0.47,1ea,eggs
4,golden valley,"born 3 white eggs, large",5.99,12 ea,$0.50/1ea,0.50,1ea,eggs
5,burnbrae farms,"naturegg omega 3 white eggs, large",6.49,12 ea,$0.54/1ea,0.54,1ea,eggs
6,golden valley,"white eggs, extra large",7.07,18 ea,$0.39/1ea,0.39,1ea,eggs
7,pc organics,free range large brown eggs,7.99,12 ea,$0.67/1ea,0.67,1ea,eggs
8,pc organics,"free-range large brown eggs, club pack (30 count)",16.99,30 ea,$0.57/1ea,0.57,1ea,eggs
9,zabiha halal,chicken sausages,6.99,375 g,$1.86/100g,1.86,100g,chicken


#### 4. Beef

In [33]:
word = 'beef'
beef_df  = food_item(word)
beef_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
8290,sapporo,"ichiban japanese style noodles, beef",0.77,100 g,$1.19/100g,1.19,100g
8542,nong shim,"cup noodle, oolongmen beef",1.29,75 g,$1.72/100g,1.72,100g
6058,no name,beef broth,1.79,900 ml,$0.20/100ml,0.20,100ml
7020,lucky me!,instant noodle soup beef,1.89,70 g,$2.70/100g,2.70,100g
3292,carl buddig,corned beef,1.99,55 g,$36.18/1kg $3.62/100g,36.18,1kg
...,...,...,...,...,...,...,...
2778,NaN,beef rib finger meat boneless,38.48,$26.43/1kg,$11.99/1lb,11.99,1lb
2930,NaN,"beef tenderloin, club pack",47.45,$79.34/1kg,$36.00/1lb,36.00,1lb
2781,NaN,"beef outside round roast, club pack",48.05,$26.43/1kg,$11.99/1lb,11.99,1lb
9617,president's choice,"certified angus beef strip loin steak, club pack",48.75,$55.09/1kg,$25.00/1lb,25.00,1lb


Taking food items having beef in `product_name` with `base_price` more than 7 as below it would consist of unuseful items

In [34]:
beef_df = beef_df[beef_df['base_price'] >= 7]
beef_df = beef_df.sort_values(by = 'base_price')

In [35]:
word_list = ['grass-fed medium ground beef', 'grass fed lean ground beef', 'free from lean ground beef', 'extra lean ground beef, club pack', 'beef tenderloin, club pack']
beef_df = include_product_rows(beef_df, word_list, 'beef')

In [36]:
from typing_extensions import final
final_df = combine_dataframes(final_df, beef_df)
final_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure,item
0,no name,"eggs, large",4.16,12 ea,$0.35/1ea,0.35,1ea,eggs
1,no name,grade a large brown eggs,5.37,12 ea,$0.45/1ea,0.45,1ea,eggs
2,burnbrae farms,naturegg simply egg whites,5.39,1 ea,$1.08/100ml,1.08,100ml,eggs
3,burnbrae farms,"naturegg nest laid white eggs, large",5.69,12 ea,$0.47/1ea,0.47,1ea,eggs
4,golden valley,"born 3 white eggs, large",5.99,12 ea,$0.50/1ea,0.50,1ea,eggs
5,burnbrae farms,"naturegg omega 3 white eggs, large",6.49,12 ea,$0.54/1ea,0.54,1ea,eggs
6,golden valley,"white eggs, extra large",7.07,18 ea,$0.39/1ea,0.39,1ea,eggs
7,pc organics,free range large brown eggs,7.99,12 ea,$0.67/1ea,0.67,1ea,eggs
8,pc organics,"free-range large brown eggs, club pack (30 count)",16.99,30 ea,$0.57/1ea,0.57,1ea,eggs
9,zabiha halal,chicken sausages,6.99,375 g,$1.86/100g,1.86,100g,chicken


#### 5. Peanuts

In [37]:
word = 'peanuts'
peanuts_df  = food_item(word)
peanuts_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
1821,t&t,chili coated peanuts,1.79,140 g,$1.28/100g,1.28,100g
3762,t&t,wasabi flavour coated peanuts,1.79,140 g,$1.28/100g,1.28,100g
744,jabsons,roasted peanuts bar-b-que,1.99,140 g,$1.42/100g,1.42,100g
153,jabsons,roasted peanuts classic salted,1.99,160 g,$1.24/100g,1.24,100g
3720,reddi bulk,praline peanuts,2.00,150 g,$1.33/100g,1.33,100g
7646,reddi bulk,honey roasted peanuts,2.00,150 g,$1.33/100g,1.33,100g
3012,kind,peanuts salted caramel nut & dark chocolate bar,2.49,40 g,$6.23/100g,6.23,100g
4074,guru lucky,chili peanuts,3.29,341 g,$0.96/100g,0.96,100g
8771,no name,salted blanched peanuts,4.49,500 g,$0.90/100g,0.90,100g
9746,president's choice,salted virginia peanuts,5.00,454 g,$1.10/100g,1.10,100g


In [38]:
word_list = ['unsalted peanuts']
peanuts_df = include_product_rows(peanuts_df, word_list, 'peanuts')

In [39]:
final_df = combine_dataframes(final_df, peanuts_df)

#### 6. Almonds

In [40]:
word = 'almonds'
almonds_df  = food_item(word)
almonds_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
5541,no name,slivered almonds,2.79,100 g,$2.79/100g,2.79,100g
3239,liberte,greek 2% vanilla bean yogurt crunch cups with ...,3.00,2x130.0 g,$1.84/100g,1.84,100g
2186,post,with almonds cereal,3.33,340 g,$1.56/100g,1.56,100g
198,president's choice,milk chocolate with almonds,3.49,100 g,$3.49/100g,3.49,100g
4857,kelloggs,special k nourish dark chocolate chunks & almo...,3.79,165 g,$2.60/100g,2.60,100g
6076,kelloggs,special k nourish cranberries & almonds bar,3.79,165 g,$2.60/100g,2.60,100g
8240,amira,natural almonds,5.00,200 g,$2.50/100g,2.50,100g
5911,president's choice,roasted almonds with sea salt,5.29,200 g,$2.65/100g,2.65,100g
9220,president's choice,hickory smoke flavour roasted almonds,5.29,200 g,$2.65/100g,2.65,100g
2498,blue diamond,"almonds, smokehouse",5.99,170 g,$3.52/100g,3.52,100g


In [41]:
word_list = ['natural almonds', 'raw california almonds']
almonds_df = include_product_rows(almonds_df, word_list, 'almonds')
final_df = combine_dataframes(final_df, almonds_df)

#### 7. Cashews

In [42]:
word = 'cashews'
cashew_df  = food_item(word)
cashew_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
2959,NaN,roasted salted cashews,3.00,$3.00/100g,$13.61/1lb,13.61,1lb
9591,president's choice,unsalted roasted cashews,5.79,200 g,$2.90/100g,2.90,100g
9225,president's choice,roasted cashews with sea salt,5.79,200 g,$2.90/100g,2.90,100g
1362,planters,cashews ranch flavoured,6.49,145 g,$4.48/100g,4.48,100g
3953,elan,organic raw cashews,6.49,185 g,$3.51/100g,3.51,100g
8475,planters,cashews salt & vinegar flavoured,6.49,145 g,$4.48/100g,4.48,100g
2690,planters,"cashews, roasted & salted",6.49,200 g,$3.25/100g,3.25,100g
5934,pc organics,whole raw cashews 200 g,6.50,200 g,$3.25/100g,3.25,100g
3490,president's choice,salted cashews almonds and macadamia nuts,7.00,200 g,$3.50/100g,3.50,100g
954,elan,coconut cashews organic,7.99,160 g,$4.99/100g,4.99,100g


In [43]:
word_list = ['unsalted roasted cashews', 'raw cashews', 'whole raw cashews']

cashew_df = include_product_rows(cashew_df, word_list, 'cashew')
final_df = combine_dataframes(final_df, cashew_df)

#### 8. Sunflower Seeds

In [44]:
word = 'sunflower seeds'
sunflower_df  = food_item(word)
sunflower_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
3484,reddi bulk,salted sunflower seeds,2.00,165 g,$1.21/100g,1.21,100g
5199,reddi bulk,"salted sunflower seeds, in shell",2.00,150 g,$1.33/100g,1.33,100g
7219,cha cha,sunflower seeds coconut flavour,2.99,260 g,$1.15/100g,1.15,100g
231,pc organics,dry-roasted salted sunflower seeds,2.99,200 g,$1.50/100g,1.50,100g
8645,spitz,dill pickle flavour sunflower seeds,3.49,210 g,$1.66/100g,1.66,100g
7374,farmer's market,salted roasted sunflower seeds,5.00,400 g,$1.25/100g,1.25,100g
7671,farmer's market,unsalted roasted sunflower seeds,5.00,400 g,$1.25/100g,1.25,100g
473,president's choice,raw sunflower seeds,5.00,350 g,$1.43/100g,1.43,100g
5193,stock and barrel,sunflower seeds roasted salted,7.00,800 g,$0.88/100g,0.88,100g


In [45]:
word_list = ['unsalted roasted sunflower seeds', 'raw sunflower seeds']
sunflower_df = include_product_rows(sunflower_df, word_list, 'sunflower seeds')
final_df = combine_dataframes(final_df, sunflower_df)

#### 9. Perch

In [46]:
word = 'perch'
perch_df  = food_item(word)
perch_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
373,NaN,ocean perch,12.29,$8.80/1kg,$3.99/1lb,3.99,1lb


In [47]:
word_list = ['ocean perch']
perch_df = include_product_rows(perch_df, word_list, 'perch')
final_df = combine_dataframes(final_df, perch_df)

#### 10. Salmon

In [48]:
word = 'salmon'
salmon_df = food_item(word)
salmon_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
8578,clover leaf,pink salmon wild pacific,2.49,142 g,$2.67/100g,2.67,100g
5690,raincoast,"wild pink salmon, no added salt",3.99,160 g,$2.49/100g,2.49,100g
8051,goldseal,wild pacific pink salmon,3.99,213 g,$2.11/100g,2.11,100g
8132,no name,sustainably sourced wild pacific pink salmon,3.99,213 g,$1.87/100g,1.87,100g
3463,president's choice,"wild pacific pink salmon, sustainably sourced",4.29,170 g,$2.52/100g,2.52,100g
2083,clover leaf,sockeye salmon wild red pacific,4.99,142 g,$3.51/100g,3.51,100g
998,goldseal,wild salmon bowl fiesta,4.99,160 g,$3.12/100g,3.12,100g
5825,clover leaf,pink salmon wild pacific,5.29,213 g,$2.48/100g,2.48,100g
9368,no name,sustainably sourced wild pacific sockeye salmon,5.49,213 g,$2.58/100g,2.58,100g
7074,goldseal,pink salmon,5.99,418 g,$1.55/100g,1.55,100g


In [49]:
word_list = ['wild pacific pink salmon', 'wild smoked pink salmon boneless']
salmon_df = include_product_rows(salmon_df, word_list, 'salmon')
final_df = combine_dataframes(final_df, salmon_df)

#### 11. Milk

In [50]:
word = 'milk'
milk_df = food_item(word)
milk_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
3344,grace,"coconut milk, powdered",0.99,50 g,$1.98/100g,1.98,100g
991,neilson,crispy crunch milkshake,1.25,310 ml,$0.51/100ml,0.51,100ml
1412,neilson,french vanilla milkshake,1.25,310 ml,$0.51/100ml,0.51,100ml
1388,maltesers,"milk chocolate candy bar, single bag",1.25,37 g,$3.38/100g,3.38,100g
7284,neilson,jersey milk chocolate,1.25,45 g,$2.78/100g,2.78,100g
...,...,...,...,...,...,...,...
4198,bio k+,kidz strawberry fermented milk,21.49,588 g,$3.65/100g,3.65,100g
5941,bio k+,"bio-k+ fermented milk probiotic, original",26.99,6x98.0 g,$4.59/100g,4.59,100g
7405,nature's way,milk thistle 60 vegetarian capsules,35.49,60 ea,$0.59/1ea,0.59,1ea
465,president's choice,club pack fully cooked buttermilk chicken brea...,35.99,1.81 kg,$1.99/100g,1.99,100g


In [51]:
milk_word_df = word_counts(milk_df)
milk_word_df.head(10)

,Word,Count
8,milk,143
9,chocolate,57
0,coconut,22
35,1%,14
19,2%,13
45,free,11
22,with,11
86,buttermilk,10
7,vanilla,10
85,&,10


In [52]:
word_list = ['chocolate','milkshake','flavour', 'coconut', 'buttermilk', 'coffee', 'vanilla', 'cheese', 'tea', 'flavor', 'cookies', 'ice']
milk_df = exclude_product_rows(milk_df, word_list)

In [53]:
word_list = ['almond milk alternative, zero sugar', 'oat milk alternative, unsweetened', 'plain goat milk yogurt']
milk_df = include_product_rows(milk_df, word_list, 'milk')
final_df = combine_dataframes(final_df, milk_df)

In [54]:
milk_df = df[df['brand'].str.contains('dairy', case=False, na=False)]
milk_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
16,dairyland,2% regular milk,5.63,4 l,$0.14/100ml,0.14,100ml
28,dairyland,3.25% milk,5.98,4 l,$0.15/100ml,0.15,100ml
549,dairyland,2% organic milk,8.99,4 l,$0.25/100ml,0.25,100ml
1003,dairyland,0% regular milk,5.62,4 l,$0.14/100ml,0.14,100ml
1036,dairyland,2% regular milk,2.21,473 ml,$0.47/100ml,0.47,100ml
1062,dairyland,1% regular milk,5.63,4 l,$0.14/100ml,0.14,100ml
1084,dairyland,2% regular milk,2.87,1 l,$0.29/100ml,0.29,100ml
1098,dairyland,3.25% protein 18 g beverage,6.99,1.89 l,$0.37/100ml,0.37,100ml
1199,dairyland,1% chocolate milk,6.80,4 l,$0.17/100ml,0.17,100ml
4432,dairyland,1% chocolate milk reduced sugar,4.32,1.89 l,$0.23/100ml,0.23,100ml


In [55]:
milk_df = milk_df[(milk_df['base_price'] == 5.63) | (milk_df['base_price'] == 5.98) | (milk_df['base_price'] == 5.62)]
milk_df = milk_df.copy()
milk_df['item'] = 'milk'
milk_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure,item
16,dairyland,2% regular milk,5.63,4 l,$0.14/100ml,0.14,100ml,milk
28,dairyland,3.25% milk,5.98,4 l,$0.15/100ml,0.15,100ml,milk
1003,dairyland,0% regular milk,5.62,4 l,$0.14/100ml,0.14,100ml,milk
1062,dairyland,1% regular milk,5.63,4 l,$0.14/100ml,0.14,100ml,milk


In [56]:
final_df = combine_dataframes(final_df, milk_df)

#### 12. Yogurt

In [57]:
word = 'yogurt'
yogurt_df = food_item(word)
yogurt_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
1100,yoplait,"yop 1% drinkable yogurt, fruity, yogurt drink ...",1.29,200 ml,$0.65/100ml,0.65,100ml
1092,yoplait,"yop 1% drinkable yogurt, blueberry, yogurt drink",1.29,200 ml,$0.65/100ml,0.65,100ml
7983,yoplait,"yop 1% drinkable yogurt, raspberry, yogurt drink",1.29,200 ml,$0.65/100ml,0.65,100ml
4581,yoplait,"yop 1% drinkable yogurt, birthday cake limited...",1.29,200 ml,$0.65/100ml,0.65,100ml
7868,yoplait,"yop 1% drinkable yogurt, mango, lactose free y...",1.29,200 ml,$0.65/100ml,0.65,100ml
...,...,...,...,...,...,...,...
1227,oikos,"greek yogurt, plain, no added sugar, 2% mf",11.99,1.5 kg,$0.80/100g,0.80,100g
749,liberte,"greek 0% yogurt 16-pack, assorted, high protei...",12.49,1.6 kg,$0.78/100g,0.78,100g
2108,olympic,no fat yogurt-plain,13.49,1.75 kg,$0.77/100g,0.77,100g
6500,olympic,"organic yogurt vanilla 3%, 1.75kg",13.49,1.75 kg,$0.77/100g,0.77,100g


In [58]:
word_list = ['yogurt with probiotics lactose free', 'lactose free greek yogurt, plain', 'organic yogurt vanilla']
yogurt_df = include_product_rows(yogurt_df, word_list, 'yogurt')
final_df = combine_dataframes(final_df, yogurt_df)

In [59]:
word_list = ['fat free greek yogurt, high protein']
yogurt_df = include_product_rows(df, word_list, 'yogurt')
final_df = combine_dataframes(final_df, yogurt_df)

#### 13. Lentil

In [60]:
word = 'lentil'
lentil_df = food_item(word)
lentil_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
5750,unico,lentils,1.99,540 ml,$0.37/100ml,0.37,100ml
1045,clic,brown lentils,2.99,907 g,$0.33/100g,0.33,100g
7478,clic,eston green lentils,2.99,907 g,$0.33/100g,0.33,100g
8047,clic,laird green lentils,2.99,907 g,$0.33/100g,0.33,100g
988,pc blue menu,curried red lentil ready-to-serve soup,3.00,500 ml,$0.60/100ml,0.60,100ml
5383,pc blue menu,lentils and quinoa blend whole grain brown rice,3.29,250 g,$1.32/100g,1.32,100g
1753,pc blue menu,green lentils,3.79,900 g,$0.42/100g,0.42,100g
7675,kitchens of india,black gram lentil curry,3.79,270 ml,$1.40/100ml,1.40,100ml
5197,pc organics,organic red lentils,4.00,450 g,$0.89/100g,0.89,100g
1446,pc organics,organic green lentils,4.00,450 g,$0.89/100g,0.89,100g


In [61]:
word_list = ['red split lentils', 'green lentils']
lentil_df = lentil_df[(lentil_df['base_price'] == 3.79) | (lentil_df['base_price'] == 5.29)]
lentil_df = include_product_rows(lentil_df, word_list, 'lentil')
final_df = combine_dataframes(final_df, lentil_df)

#### 14. Chickpeas

In [62]:
word = 'chickpeas'
chickpeas_df = food_item(word)
chickpeas_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
5159,suraj,chickpeas,1.29,540 ml,$0.28/100ml,0.28,100ml
3686,tamam,lebanese chickpeas,1.49,400 g,$0.37/100g,0.37,100g
9394,suraj,chickpeas,3.50,907 g,$0.39/100g,0.39,100g
9141,pc blue menu,chickpeas,3.79,900 g,$0.42/100g,0.42,100g
5626,pataks,butter chickpeas ready to eat,4.29,285,ml,NaN,NaN
4190,three farmers,roasted chickpeas sea salt & lime,4.99,120 g,$4.16/100g,4.16,100g
8078,suraj,chickpeas,5.49,1.8 kg,$0.31/100g,0.31,100g
581,quality,chickpeas,6.99,1.8 kg,$0.39/100g,0.39,100g


In [63]:
word_list = ['chickpea']
chickpeas_df = chickpeas_df[(chickpeas_df['base_price'] == 5.49)]
chickpeas_df = include_product_rows(chickpeas_df, word_list, 'chickpeas')
final_df = combine_dataframes(final_df, chickpeas_df)

####15. Beans

In [64]:
word = 'beans'
beans_df = food_item(word)
beans_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
3398,NaN,green beans,1.10,$1.10/100g,$11.00/1kg,11.00,1kg
327,tamam,pinto beans,1.29,540 ml,$0.28/100ml,0.28,100ml
1614,tamam,faba beans foul medamas,1.29,540 ml,$0.28/100ml,0.28,100ml
8463,tamam,small red beans,1.29,540 ml,$0.28/100ml,0.28,100ml
8611,tamam,white kidney beans,1.29,540 ml,$0.28/100ml,0.28,100ml
...,...,...,...,...,...,...,...
4066,jelly belly,"jelly beans, assorted flavours",7.29,255 g,$2.86/100g,2.86,100g
3925,quality,"red kidney beans, dark",7.99,1.81 kg,$0.44/100g,0.44,100g
7362,no name,baked beans in tomato sauce,17.00,12x398.0 ml,$0.36/100ml,0.36,100ml
7093,quality,blackeye beans,21.99,5 kg,$0.44/100g,0.44,100g


In [65]:
word_list = ['light red kidney beans', 'pinto beans']
beans_df = beans_df[(beans_df['base_price'] == 1.29) | (beans_df['base_price'] == 5.49)]
beans_df = include_product_rows(beans_df, word_list, 'beans')
final_df = combine_dataframes(final_df, beans_df)

#### 15. Tofu

In [66]:
word = 'tofu'
tofu_df = food_item(word)
tofu_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure
776,sunrise,"tofu dessert, original",2.29,2x150.0 g,$0.76/100g,0.76,100g
3470,soyganic,tofu soft organic,2.29,300 g,$0.76/100g,0.76,100g
8413,sunrise,"tofu dessert, peach mango",2.29,2x150.0 g,$0.76/100g,0.76,100g
9233,sunrise,"tofu, medium firm",2.47,454 g,$0.54/100g,0.54,100g
3764,rooster,silken tofu,2.49,454 g,$0.55/100g,0.55,100g
4271,sunrise,premium soft tofu,2.79,530 g,$0.53/100g,0.53,100g
30,sunrise,"tofu, extra firm",3.49,350 g,$1.00/100g,1.00,100g
9305,soyganic,tofu firm organic 500 g,3.79,500 g,$0.76/100g,0.76,100g
206,soyganic,medium firm organic tofu,3.99,454 g,$0.88/100g,0.88,100g
8243,kikkoman,"instant miso soup, tofu",3.99,30 g,$13.30/100g,13.30,100g


In [67]:
word_list = ['tofu']
tofu_df = tofu_df[(tofu_df['product_name'] == 'premium soft tofu') | (tofu_df['base_price'] == 2.47)]
tofu_df = include_product_rows(tofu_df, word_list, 'tofu')
final_df = combine_dataframes(final_df, tofu_df)

In [68]:
final_df

,brand,product_name,base_price,product_size,price_per_unit,unit_price_value,unit_measure,item
0,no name,"eggs, large",4.16,12 ea,$0.35/1ea,0.35,1ea,eggs
1,no name,grade a large brown eggs,5.37,12 ea,$0.45/1ea,0.45,1ea,eggs
2,burnbrae farms,naturegg simply egg whites,5.39,1 ea,$1.08/100ml,1.08,100ml,eggs
3,burnbrae farms,"naturegg nest laid white eggs, large",5.69,12 ea,$0.47/1ea,0.47,1ea,eggs
4,golden valley,"born 3 white eggs, large",5.99,12 ea,$0.50/1ea,0.50,1ea,eggs
5,burnbrae farms,"naturegg omega 3 white eggs, large",6.49,12 ea,$0.54/1ea,0.54,1ea,eggs
6,golden valley,"white eggs, extra large",7.07,18 ea,$0.39/1ea,0.39,1ea,eggs
7,pc organics,free range large brown eggs,7.99,12 ea,$0.67/1ea,0.67,1ea,eggs
8,pc organics,"free-range large brown eggs, club pack (30 count)",16.99,30 ea,$0.57/1ea,0.57,1ea,eggs
9,zabiha halal,chicken sausages,6.99,375 g,$1.86/100g,1.86,100g,chicken


In [69]:
final_df.to_csv("food_collection.csv",index=False)